In [83]:
import pandas as pd

# Path to your file (adjust if your filename differs)
file_path = "/Users/lucywu/Documents/Gong/Copy of Copy of Data for VE Exercise - Candidate Facing - Lucy Wu.xlsx"

# Load the Excel file
dfs = pd.read_excel(file_path, sheet_name=None)

df_performance = dfs["Performance Data (static)"]
df_sales = dfs["Gong Data (static)"]

df = df_performance.merge(df_sales, on = "Rep")

df = df[(df["Deal size Q3"] * df["Number of deals Q3"] == df["Revenue Per Rep Q3"]) & (df["Deal size Q4"] * df["Number of deals Q4"] == df["Revenue Per Rep Q4"])]
df = df[(df["Time to first deal (days)"] >= 0) | (df["Time to first deal (days)"].isnull())]



In [ ]:
#new columns for analysis
df["num_deals_growth"] = (df["Number of deals Q4"] - df["Number of deals Q3"])/df["Number of deals Q3"]
df["rev_per_rep_growth"] = (df["Revenue Per Rep Q4"] - df["Revenue Per Rep Q3"])/df["Revenue Per Rep Q3"]
df["deal_size_growth"] = (df["Deal size Q4"] - df["Deal size Q3"])/df["Deal size Q3"]

df["pct_new_pitch"] = df["# times rep used new sales pitch"]/(df["# times rep used new sales pitch"] + df["# times rep used old sales pitch"])

df["calls_listened_by_manager_scaled"] = ((df["# rep's calls listened by their manager"] - df["# rep's calls listened by their manager"].min()) /(df["# rep's calls listened by their manager"].max() - df["# rep's calls listened by their manager"].min())) * 100
df["peer_calls_listened_scaled"] = ((df["# peer calls rep listened to"] - df["# peer calls rep listened to"].min()) /(df["# peer calls rep listened to"].max() - df["# peer calls rep listened to"].min())) * 100
df["rep_accessed_deal_board_scaled"] = ((df["# times rep accessed the deal board"] - df["# times rep accessed the deal board"].min()) /(df["# times rep accessed the deal board"].max() - df["# times rep accessed the deal board"].min())) * 100
df["num_deals_growth_scaled"] = ((df["num_deals_growth"] - df["num_deals_growth"].min()) /(df["num_deals_growth"].max() - df["num_deals_growth"].min())) * 100

df["usage_score"] = (0.5*df["peer_calls_listened_scaled"])  + (0.12*df["calls_listened_by_manager_scaled"]) + (0.38*df["rep_accessed_deal_board_scaled"])

df.loc[(df['usage_score'] >= 70), 'Usage_Bucket'] = 'high'
df.loc[(df['usage_score'] < 70) & (df['usage_score'] >= 50), 'Usage_Bucket'] = 'medium'
df.loc[(df['usage_score'] < 50), 'Usage_Bucket'] = 'low'

for col in ["num_deals_growth", "rev_per_rep_growth", "deal_size_growth"]:
    df[col + "_score"] = df[col].rank(pct=True) * 100

df.to_csv("/Users/lucywu/Documents/Gong/lucy_edited.csv")

In [115]:
import statsmodels.api as sm

# Select variables
X_rev_per_rep = df[["calls_listened_by_manager_scaled", "peer_calls_listened_scaled", "rep_accessed_deal_board_scaled"]]
y_rev_per_rep = df["rev_per_rep_growth"]

# Add intercept
X_rev_per_rep = sm.add_constant(X_rev_per_rep)

# Fit regression
model = sm.OLS(y_rev_per_rep, X_rev_per_rep).fit()

# View results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:     rev_per_rep_growth   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     55.40
Date:                Wed, 19 Nov 2025   Prob (F-statistic):           4.19e-33
Time:                        16:51:11   Log-Likelihood:                 380.52
No. Observations:                 994   AIC:                            -753.0
Df Residuals:                     990   BIC:                            -733.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

In [114]:
import statsmodels.api as sm

# Select variables
X_deal_size = df[["calls_listened_by_manager_scaled", "peer_calls_listened_scaled", "rep_accessed_deal_board_scaled"]]
y_deal_size = df["deal_size_growth"]

# Add intercept
X_deal_size = sm.add_constant(X_deal_size)

# Fit regression
model = sm.OLS(y_deal_size, X_deal_size).fit()

# View results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:       deal_size_growth   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.168
Method:                 Least Squares   F-statistic:                     67.82
Date:                Wed, 19 Nov 2025   Prob (F-statistic):           6.90e-40
Time:                        16:50:13   Log-Likelihood:                 502.75
No. Observations:                 994   AIC:                            -997.5
Df Residuals:                     990   BIC:                            -977.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

In [110]:
#Correlations separated by new hire and tenured hire

new_hire_df = df[df['New Hire'] == 'Yes']
tenured_hire_df = df[df['New Hire'] == 'No']

correl_new_hire = new_hire_df.select_dtypes(include="number").corr()[['num_deals_growth', 'rev_per_rep_growth', 'deal_size_growth']]
correl_tenured_hire = tenured_hire_df.select_dtypes(include="number").corr()[['num_deals_growth', 'rev_per_rep_growth', 'deal_size_growth']]
correl = df.select_dtypes(include="number").corr()[['num_deals_growth', 'rev_per_rep_growth', 'deal_size_growth']]

print(correl)


                                                  num_deals_growth  \
Rep                                                       0.056034   
Deal size Q3                                              0.004282   
Deal size Q4                                             -0.011970   
Number of deals Q3                                        0.000865   
Number of deals Q4                                        0.692742   
Revenue Per Rep Q3                                        0.005712   
Revenue Per Rep Q4                                        0.313986   
Time to first deal (days)                                -0.108968   
# rep's calls listened by their manager                   0.049573   
# peer calls rep listened to                             -0.004223   
# times rep accessed the deal board                      -0.005915   
Interactivity score                                       0.014635   
Longest monologue (min)                                   0.030011   
# times competitor w

In [112]:
#slowest 25 percentile of users to onboard

new_hire_threshold = new_hire_df['Time to first deal (days)'].quantile(0.75)
new_hire_bottom_25 = new_hire_df[new_hire_df['Time to first deal (days)'] >= new_hire_threshold]

correl_new_hire_bottom_25 = new_hire_bottom_25.select_dtypes(include="number").corr()[['num_deals_growth', 'rev_per_rep_growth', 'deal_size_growth']]

print(new_hire_bottom_25["Usage_Bucket"].value_counts())
print(new_hire_bottom_25.groupby('Usage_Bucket').mean(numeric_only=True)[['Revenue Per Rep Q3','Revenue Per Rep Q4', "rev_per_rep_growth","pct_new_pitch"]])

new_hire_bottom_25.to_excel("/Users/lucywu/Documents/Gong/lucy_edited_bottom_25_new.xlsx")





Usage_Bucket
low       51
medium    10
high       4
Name: count, dtype: int64
              Revenue Per Rep Q3  Revenue Per Rep Q4  rev_per_rep_growth  \
Usage_Bucket                                                               
high               114572.144792       121494.473872            0.059650   
low                 89524.541751        78934.333925           -0.127103   
medium             104081.560677       106445.585189            0.022412   

              pct_new_pitch  
Usage_Bucket                 
high               0.651286  
low                0.391648  
medium             0.542286  


In [113]:
new_hire_threshold_twentyfive = new_hire_df['Time to first deal (days)'].quantile(0.25)
print(new_hire_threshold_twentyfive)
new_hire_top_25 = new_hire_df[new_hire_df['Time to first deal (days)'] <= new_hire_threshold_twentyfive]

#correl_new_hire_bottom_25 = new_hire_bottom_25.select_dtypes(include="number").corr()[['num_deals_growth', 'rev_per_rep_growth', 'deal_size_growth']]

print(new_hire_top_25["Usage_Bucket"].value_counts())
print(new_hire_top_25.groupby('Usage_Bucket').mean(numeric_only=True)[['Revenue Per Rep Q3','Revenue Per Rep Q4','num_deals_growth', "rev_per_rep_growth","pct_new_pitch"]])
print(new_hire_top_25['Revenue Per Rep Q4'].mean())

#new_hire_bottom_25.to_excel("/Users/lucywu/Documents/Gong/lucy_edited_bottom_25_new.xlsx")




27.0
Usage_Bucket
medium    30
high      22
low       10
Name: count, dtype: int64
              Revenue Per Rep Q3  Revenue Per Rep Q4  num_deals_growth  \
Usage_Bucket                                                             
high               112124.446638       115859.675704         -0.025023   
low                109582.928095       114974.118142          0.014072   
medium             109246.385149       117165.554078          0.012274   

              rev_per_rep_growth  pct_new_pitch  
Usage_Bucket                                     
high                    0.034999       0.568520  
low                     0.059161       0.557636  
medium                  0.074025       0.642266  
116348.72047222468
